In [ ]:
import mlflow
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
os.environ['AWS_ACCESS_KEY_ID'] = 'minio'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'password'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
os.environ['MLFLOW_S3_ENDPOINT_URL']='http://localhost:9000'

i=8

In [ ]:
mlflow.set_tracking_uri('http://localhost:5000')
i=i+1
mlflow.set_experiment('my-experiment448' + str(i))

mlflow.start_run()
try:
    # log a parameter
    mlflow.log_param('my-param', 14)
    mlflow.set_tag('tag', 'mytag')
    # log several metrics
    for i in range(0,5):
        mlflow.log_metric('metric', i, step=i)
        mlflow.log_metric('metric_squared', i*i, step=i)

    # log a figure
    fig, ax = plt.subplots()
    x=np.array(range(-10,11))
    subfig = sns.lineplot(x=x,y=x*x) 
    mlflow.log_figure(fig, "myfigure.png")
 
     
finally:
    mlflow.end_run()


In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import mlflow.sklearn
from mlflow.models.signature import infer_signature

In [ ]:
experiment_id = mlflow.set_experiment('lfr-demo-model5')
mlflow.start_run()
try:
    iris = datasets.load_iris()
    iris_train = pd.DataFrame(iris.data, columns=iris.feature_names)
    mlflow.sklearn.autolog() 
    clf = RandomForestClassifier(max_depth=7, random_state=0)
    clf.fit(iris_train, iris.target)
    signature = infer_signature(iris_train, clf.predict(iris_train))
    mlflow.sklearn.log_model(clf, "iris_rf", signature=signature)
finally:
    mlflow.end_run()
